In [1]:
!pip install boto3

In [2]:
from pyspark.sql import functions as f
from pyspark.sql.window import Window as W
from pyspark.sql import SparkSession
import boto3

## 1 - Configurando spark

In [3]:
spark = SparkSession.builder \
    .appName("Teste PySpark") \
    .master("spark://spark-master:7077") \
    .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:3.3.4") \
    .getOrCreate()

In [4]:
hadoop_conf = spark._jsc.hadoopConfiguration()
hadoop_conf.set("fs.s3a.endpoint", "http://localstack:4566")
hadoop_conf.set("fs.s3a.access.key", "test")
hadoop_conf.set("fs.s3a.secret.key", "test")
hadoop_conf.set("fs.s3a.path.style.access", "true")
hadoop_conf.set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")


In [5]:
# data = [("Alice", 25), ("Bob", 30), ("Carol", 27)]
# df = spark.createDataFrame(data, ["Nome", "Idade"])
# df.show()

## 2 - Acessando bucket S3

In [6]:
s3 = boto3.client(
    "s3",
    endpoint_url="http://localstack:4566",
    aws_access_key_id="test",
    aws_secret_access_key="test",
    region_name="us-east-1"
)

bucket_name = "s3-cvm-fii"
prefix = "raw/"

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

files = [f"s3a://{bucket_name}/{obj['Key']}" for obj in response.get("Contents", []) if obj['Key'].endswith('.csv')]

# for path in files:
#     print(path)
print('ok \u2705')


ok ✅


## 3 - lendo arquivos do bucket e unificando

In [7]:
try:
    df = spark.read \
        .option("header", "true") \
        .option("encoding", "latin1") \
        .option("sep", ";") \
        .option("inferSchema", "true") \
        .csv("s3a://s3-cvm-fii/raw/*.csv")
    print('ok \u2705')
except Exception as e:
    print(f'\u270c{e}')

ok ✅


In [8]:
df.show(10,truncate=30)

+---------------+------------------+------------+----------+----------+---------------+-------------+---------+--------+--------+
|TP_FUNDO_CLASSE| CNPJ_FUNDO_CLASSE|ID_SUBCLASSE| DT_COMPTC|  VL_TOTAL|       VL_QUOTA|VL_PATRIM_LIQ|CAPTC_DIA|RESG_DIA|NR_COTST|
+---------------+------------------+------------+----------+----------+---------------+-------------+---------+--------+--------+
|  CLASSES - FIF|00.017.024/0001-53|        NULL|2025-07-01|1139381.41|39.213324800000|   1146691.57|     0.00|    0.00|       1|
|  CLASSES - FIF|00.017.024/0001-53|        NULL|2025-07-02|1140016.83|39.232973700000|   1147266.15|     0.00|    0.00|       1|
|  CLASSES - FIF|00.017.024/0001-53|        NULL|2025-07-03|1140651.92|39.252822600000|   1147846.58|     0.00|    0.00|       1|
|  CLASSES - FIF|00.017.024/0001-53|        NULL|2025-07-04|1141271.40|39.271309800000|   1148387.19|     0.00|    0.00|       1|
|  CLASSES - FIF|00.017.024/0001-53|        NULL|2025-07-07|1137959.70|39.289262100000|   

In [9]:
df.count()

22390517

In [10]:
df.printSchema()

root
 |-- TP_FUNDO_CLASSE: string (nullable = true)
 |-- CNPJ_FUNDO_CLASSE: string (nullable = true)
 |-- ID_SUBCLASSE: string (nullable = true)
 |-- DT_COMPTC: string (nullable = true)
 |-- VL_TOTAL: string (nullable = true)
 |-- VL_QUOTA: string (nullable = true)
 |-- VL_PATRIM_LIQ: string (nullable = true)
 |-- CAPTC_DIA: string (nullable = true)
 |-- RESG_DIA: string (nullable = true)
 |-- NR_COTST: integer (nullable = true)



In [11]:
columns_rename = {
                    "TP_FUNDO_CLASSE": "tipo_fundo",
                     "CNPJ_FUNDO_CLASSE": "cnpj_fundo",
                     "DT_COMPTC": "data_referencia",
                     "NR_COTST": "qtd_cotistas",
                     "RESG_DIA": "valor_resgates",
                     "CAPTC_DIA": "valor_aplicacoes",
                     "VL_QUOTA": "cota",
                     "VL_TOTAL": "valor_carteira",
                     "VL_PATRIM_LIQ":'pl_fundo'
                    }

for old_name, new_name in columns_rename.items():
    df = df.withColumnRenamed(old_name, new_name)


In [12]:
df_fi=df.filter(f.col('TP_FUNDO_CLASSE')=='FI')

In [13]:
df_fi = (
        df_fi
        .withColumn('cnpj_fundo',f.regexp_replace(f.col('cnpj_fundo'), r'[./-]', ''))
        .select(
             'cnpj_fundo',
             'pl_fundo',
             'cota',
             'qtd_cotistas',
             'valor_aplicacoes',
             'valor_resgates',
             'valor_carteira',
             'data_referencia')
        )

In [14]:
df_teste = df_fi.limit(10)
df_teste.show()

+--------------+------------+------------------+------------+----------------+--------------+--------------+---------------+
|    cnpj_fundo|    pl_fundo|              cota|qtd_cotistas|valor_aplicacoes|valor_resgates|valor_carteira|data_referencia|
+--------------+------------+------------------+------------+----------------+--------------+--------------+---------------+
|00180995000110| 85879995.16|  833.451043000000|         580|            0.00|     391109.66|   85851492.09|     2025-07-01|
|00185259000154|127705173.61|   71.957563520000|          46|            0.00|          0.00|  128006157.89|     2025-07-01|
|00222816000160|691540464.19|12093.294198000000|        1198|          267.53|     121657.11|  691496484.62|     2025-07-01|
|00539553000117|320571707.81|  499.408022000000|        4318|       614862.13|    3430064.69|  321269487.04|     2025-07-01|
|00601692000123| 65893995.77| 6798.839805910000|         794|         1000.00|      10428.54|   66061379.65|     2025-07-01|


In [15]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [17]:
## variação cota dia
df_fi = (
    df_fi.withColumn('cota_dia_anterior',
                         f.lag(f.col('cota')).over(W.partitionBy(f.col('cnpj_fundo'))
                                                     .orderBy(f.col('data_referencia'))))
        .withColumn("variacao_cota_dia",
            f.when(
                   (f.col("cota_dia_anterior").isNotNull()) & (f.col("cota_dia_anterior") != 0),
                    f.round(((f.col("cota") - f.col("cota_dia_anterior")) / f.col("cota_dia_anterior")) * 100,4)))
        .withColumn("ano", f.year(f.col("data_referencia")))
        .withColumn("mes",f.month(f.col("data_referencia")))
        .withColumn("net",
                   f.col("valor_aplicacoes") - f.col("valor_resgates"))
        .withColumn("pl_d1",
                    f.lag(f.col("pl_fundo")).over(W.partitionBy(f.col("cnpj_fundo"))
                                                    .orderBy(f.col("data_referencia")))
                    )
        .withColumn('pnl',f.col('pl_fundo') - f.col('pl_d1') - f.col('net'))
        .withColumn("dt_ingest", f.current_date())
        .select(
             'cnpj_fundo',
             'pl_fundo',
             'cota',
             'qtd_cotistas',
             'valor_aplicacoes',
             'valor_resgates',
             'net',
             'pnl',
             'valor_carteira',
             'data_referencia',
             'variacao_cota_dia',
             'data_referencia',
             'mes',
             'ano',
            'dt_ingest')
             
).orderBy('data_referencia')

In [18]:
df_fi.write.mode("overwrite").parquet("s3a://s3-cvm-fii/stage/")


Py4JJavaError: An error occurred while calling o151.parquet.
: org.apache.spark.SparkException: Job aborted due to stage failure: ResultStage 14 (parquet at NativeMethodAccessorImpl.java:0) has failed the maximum allowable number of times: 4. Most recent failure reason:
org.apache.spark.shuffle.MetadataFetchFailedException: Missing an output location for shuffle 2 partition 0
	at org.apache.spark.MapOutputTracker$.validateStatus(MapOutputTracker.scala:1739)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11(MapOutputTracker.scala:1686)
	at org.apache.spark.MapOutputTracker$.$anonfun$convertMapStatuses$11$adapted(MapOutputTracker.scala:1685)
	at scala.collection.Iterator.foreach(Iterator.scala:943)
	at scala.collection.Iterator.foreach$(Iterator.scala:943)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1431)
	at org.apache.spark.MapOutputTracker$.convertMapStatuses(MapOutputTracker.scala:1685)
	at org.apache.spark.MapOutputTrackerWorker.getMapSizesByExec

In [ ]:
print('k')